# Clustering the Neigborhoods of London and Paris

Tiffany Kwan

## 1. Introduction

London and Paris are two of the most popular tourist destinations.  This project will group the two neighborhoods respectively to draw insights into the two cities.

## 2. Business Problem

We want to help tourists choose which neighborhood to visit based on likes and experiences.  Not only will this help tourists, but it will also help individuals who are considering relocating to one of these neigborhoods.  Further, this will help stakeholders makw decisions based on what amenities wach city has to offer.

## 3. Data 

We will use postal codes as a starting point for geographical location data in London and Paris.  The codes will help us find popular venues, neighborhoods and boroughs. 

### London

We will use data from https://en.wikipedia.org/wiki/List_of_areas_of_London

For more information about geographical locations, we will utilize ArcGIS API which will enable us to connect people, locations and data using interactive maps. 

The following will be the variables used

1. borough: Name of Neighborhood
2. town: Name of borough
3. post_code: postal codes for London
4. latitiude: Latitdue for Neighborhood
5. longitude: Longitde for Neighborhood

### Paris

We will use data from JSON data https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

The following variables will be used

1. postal_code: postal codes for France
2. nom_comm: Name of Neighborhood in France
3. nom_dept: Name of the boroughs
4. feo_point_2d: Tuple containing the latitude and longitude

#### Foursquare API Data

We will use Foursquare data to find data about specific venues in different neighnorhoods.  First, we will find the list of neighborhoods, then connect to Foursquare API Data to find venues inside each neighborhood within a 500 meter radius.  

The following variables will be used

1. Neighbourhood : Name of the Neighbourhood
2. Neighbourhood Latitude : Latitude of the Neighbourhood
3. Neighbourhood Longitude : Longitude of the Neighbourhood
4. Venue : Name of the Venue
5. Venue Latitude : Latitude of Venue
6. Venue Longitude : Longitude of Venue
7. Venue Category : Category of Venue

All of this collected data will help us build our model 

## Methodology

Import all the required packages

In [1]:
!pip install folium

     |████████████████████████████████| 93 kB 4.2 MB/s  eta 0:00:01


In [2]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans

These packages will help us explore each city individually. Also, we would be able to plot the map to show the separate neighborhoods, then build a model based on similar characteristics.  

## Exploring London

Start by obtaining the list of areas of london

In [3]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

We were able to make the connection

In [4]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 527         Woolwich                          Greenwich          LONDON   
 528   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 529  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 530          Yeading                         Hillingdon           HAYES   
 

We need to select the second table

In [5]:
wiki_london_data = wiki_london_data[1]
wiki_london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
527,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
528,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
529,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
530,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


## Data Preprocessing

Remove the spaces and add _ in between words

In [6]:
wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_london_data

,Location,London borough,Post_town,Postcode district,Dial code,OS_grid_ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
527,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
528,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
529,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
530,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


Some columns contain special characters so there is not _ between the words.

## Feature Selection

We can drop locations, dial codes and OS grid

In [7]:
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)

In [8]:
df1.head()


,London borough,Post_town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


Rename the Postcode district column

In [9]:

df1.columns = ['borough','town','post_code']
df1

,borough,town,post_code
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
527,Greenwich,LONDON,SE18
528,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
529,Hammersmith and Fulham,LONDON,W12
530,Hillingdon,HAYES,UB4


Remove the brackets

In [10]:
df1['borough'] = df1['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
527,Greenwich,LONDON,SE18
528,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
529,Hammersmith and Fulham,LONDON,W12
530,Hillingdon,HAYES,UB4


Dimensions of the dataframe:

In [11]:
df1.shape


(532, 3)

We have 533 records and 3 columns of data

## Feature Engineering

Currently, we want to focus on the neighborhoods of London

In [12]:

df1 = df1[df1['town'].str.contains('LONDON')]
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20
...,...,...,...
522,Redbridge,LONDON,"IG8, E18"
523,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8
526,Barnet,LONDON,N12
527,Greenwich,LONDON,SE18


In [13]:
df1.shape

(309, 3)

Now with 309 rows, we will try to get some descriptive statistics

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 529
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   borough    309 non-null    object
 1   town       309 non-null    object
 2   post_code  309 non-null    object
dtypes: object(3)
memory usage: 9.7+ KB


## Geolocations of the London Neighborhoods

### ArcGis API

Find the geographical coordinates for the enighborhoods in order to plot out the map

In [15]:
!pip install arcgis

In [16]:

from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


Define London arcgis to return latitude and longitude

In [17]:

def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England, GBR'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [18]:
c = get_x_y_uk('SE2')

In [19]:
c

'51.492450000000076,0.12127000000003818'

Copy over the postal codes and pus it to the geolocator function

In [20]:
geo_coordinates_uk = df1['post_code']    
geo_coordinates_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
522    IG8, E18
523         IG8
526         N12
527        SE18
529         W12
Name: post_code, Length: 309, dtype: object

Pass the postal codes to get geographical coordinates

In [21]:

coordinates_latlng_uk = geo_coordinates_uk.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_uk

0       51.492450000000076,0.12127000000003818
1        51.51324000000005,-0.2674599999999714
6       51.51200000000006,-0.08057999999994081
7       51.51651000000004,-0.11967999999995982
9       51.41009000000008,-0.05682999999993399
                        ...                   
522    51.589770000000044,0.030520000000024083
523      51.50642000000005,-0.1272099999999341
526     51.615920000000074,-0.1767399999999384
527      51.48207000000008,0.07143000000002075
529      51.50645000000003,-0.2369099999999662
Name: post_code, Length: 309, dtype: object

### Latitude

Extract the Latitude

In [22]:
lat_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[0])
lat_uk

0      51.492450000000076
1       51.51324000000005
6       51.51200000000006
7       51.51651000000004
9       51.41009000000008
              ...        
522    51.589770000000044
523     51.50642000000005
526    51.615920000000074
527     51.48207000000008
529     51.50645000000003
Name: post_code, Length: 309, dtype: object

### Longitude

Extract the Longitude

In [23]:
lng_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[1])
lng_uk

0       0.12127000000003818
1       -0.2674599999999714
6      -0.08057999999994081
7      -0.11967999999995982
9      -0.05682999999993399
               ...         
522    0.030520000000024083
523     -0.1272099999999341
526     -0.1767399999999384
527     0.07143000000002075
529     -0.2369099999999662
Name: post_code, Length: 309, dtype: object

Now that we have the geographical coordinates of London's neighborhoods, we can merger our source data with the coordinates

In [24]:
london_merged = pd.concat([df1,lat_uk.astype(float), lng_uk.astype(float)], axis=1)
london_merged.columns= ['borough','town','post_code','latitude','longitude']
london_merged


,borough,town,post_code,latitude,longitude
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746
6,City,LONDON,EC3,51.51200,-0.08058
7,Westminster,LONDON,WC2,51.51651,-0.11968
9,Bromley,LONDON,SE20,51.41009,-0.05683
...,...,...,...,...,...
522,Redbridge,LONDON,"IG8, E18",51.58977,0.03052
523,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.50642,-0.12721
526,Barnet,LONDON,N12,51.61592,-0.17674
527,Greenwich,LONDON,SE18,51.48207,0.07143


In [25]:
london_merged.dtypes


borough       object
town          object
post_code     object
latitude     float64
longitude    float64
dtype: object

### Coordinates of London

In [26]:
london = geocode(address='London, England, GBR')[0]
london_lng_coords = london['location']['x']
london_lat_coords = london['location']['y']
london_lng_coords

-0.1272099999999341

In [27]:
london_lat_coords


51.50642000000005

This is london's geocode

### Visualize the Map of London

Use the folium package to visualize the Map of London

In [28]:
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_London

# adding markers to map
for latitude, longitude, borough, town in zip(london_merged['latitude'], london_merged['longitude'], london_merged['borough'], london_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

### Venues in London

We will now use FOursquare API to get the venue and venue categories around each neighborhood in London

In [29]:
CLIENT_ID = 'BLN5212LPUOTLJ5ZQGYTKZBR2PQYQTOAKSAS4R15KALGEE0D' 
CLIENT_SECRET = 'PPJXW2LHVZ0HFJGLTTOFZ43KYMVDAC3WYMEYRLN4PY1O2WR4'
VERSION = '20180605' # Foursquare API version

Now, obtain venue categories

In [30]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues in London

In [31]:
venues_in_London = getNearbyVenues(london_merged['borough'], london_merged['latitude'], london_merged['longitude'])



Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and ChelseaHammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames
En

Now, it is time to sample data

In [32]:
venues_in_London.head()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Lesnes Abbey,Historic Site
1,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,Supermarket
2,"Bexley, Greenwich",51.49245,0.12127,Lidl,Supermarket
3,"Bexley, Greenwich",51.49245,0.12127,Abbey Wood Railway Station (ABW),Train Station
4,"Bexley, Greenwich",51.49245,0.12127,Bean @ Work,Coffee Shop


In [33]:
venues_in_London.shape


(10882, 5)

### Grouping by Venue Categories

In [34]:
venues_in_London.groupby('Venue Category').max()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Adult Boutique,Islington,51.52969,-0.08697,Sh! Women's Erotic Emporium
African Restaurant,Westminster,51.52587,-0.00754,Red Sea Restaurant
American Restaurant,Waltham Forest,51.63261,0.02795,Spielburger
Antique Shop,Westminster,51.51651,-0.11968,The London Silver Vaults
Arcade,Westminster,51.51651,-0.11968,Novelty Automation
...,...,...,...,...
Wings Joint,Hammersmith and Fulham,51.54187,-0.19795,Wingmans
Women's Store,Kensington and ChelseaHammersmith and Fulham,51.55457,-0.11478,Vivien of Holloway
Xinjiang Restaurant,Southwark,51.47480,-0.09313,Silk Road


This area is very diverse: 299 records

### One Hot Encoding

Time to encode venue categories in order to cluster

In [35]:
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat


,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Venue categories mean value

In [37]:
London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()


,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,Barnet,0.0,0.0,0.006536,0.0,0.0,0.0,0.006536,0.0,0.0,...,0.0,0.006536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Barnet, Brent, Camden",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bexley,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

### Top Venue Categories

In [40]:

# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Café,Grocery Store,Pub,Italian Restaurant,Turkish Restaurant,Bus Stop,Supermarket,Pharmacy,Sushi Restaurant
1,"Barnet, Brent, Camden",Music Venue,Bus Station,Clothing Store,Gym / Fitness Center,Supermarket,Historic Site,Fishing Store,Fast Food Restaurant,Filipino Restaurant,Film Studio
2,Bexley,Supermarket,Park,Historic Site,Pub,Train Station,Coffee Shop,Convenience Store,Bus Stop,Construction & Landscaping,Golf Course
3,"Bexley, Greenwich",Sports Club,Convenience Store,Golf Course,Park,Historic Site,Construction & Landscaping,Bus Stop,Flower Shop,Flea Market,Fishing Store
4,"Bexley, Greenwich",Supermarket,Pub,Park,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Falafel Restaurant,Farmers Market


### Model Building

Use K Means clustering to cluster the city of London to 5

In [41]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(n_clusters=5, random_state=0)

In [42]:
kmeans_london.labels_


array([0, 4, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)

In [43]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

In [44]:
london_data = london_merged

london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')

london_data.head()

,borough,town,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127,4,Supermarket,Pub,Park,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Falafel Restaurant,Farmers Market
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746,2,Grocery Store,Indian Restaurant,Train Station,Breakfast Spot,Park,Zoo Exhibit,Fish & Chips Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
6,City,LONDON,EC3,51.51200,-0.08058,1,Hotel,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Restaurant,Sandwich Place,Wine Bar,Cocktail Bar,Falafel Restaurant
7,Westminster,LONDON,WC2,51.51651,-0.11968,1,Hotel,Coffee Shop,Café,Sandwich Place,Pub,Italian Restaurant,Theater,Restaurant,French Restaurant,Burger Joint
9,Bromley,LONDON,SE20,51.41009,-0.05683,1,Supermarket,Fish & Chips Shop,Hotel,Fast Food Restaurant,Grocery Store,Convenience Store,Park,Portuguese Restaurant,Construction & Landscaping,Caribbean Restaurant


Drop the NaN values to prevent data skew

In [45]:
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])


### Visualizing clustered neighborhood

In [46]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['latitude'], london_data_nonan['longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london

### Examining our clusters

Cluster 1

In [47]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,LONDON,1,Hotel,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Restaurant,Sandwich Place,Wine Bar,Cocktail Bar,Falafel Restaurant
7,LONDON,1,Hotel,Coffee Shop,Café,Sandwich Place,Pub,Italian Restaurant,Theater,Restaurant,French Restaurant,Burger Joint
9,LONDON,1,Supermarket,Fish & Chips Shop,Hotel,Fast Food Restaurant,Grocery Store,Convenience Store,Park,Portuguese Restaurant,Construction & Landscaping,Caribbean Restaurant
10,LONDON,1,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Gym / Fitness Center,Italian Restaurant,Park,Cocktail Bar,Breakfast Spot
12,LONDON,1,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Gym / Fitness Center,Italian Restaurant,Park,Cocktail Bar,Breakfast Spot
...,...,...,...,...,...,...,...,...,...,...,...,...
522,LONDON,1,Café,Pub,Grocery Store,Coffee Shop,Bakery,Metro Station,Pharmacy,BBQ Joint,Park,Seafood Restaurant
523,"LONDON, WOODFORD GREEN",1,Café,Hotel,Pub,Theater,Monument / Landmark,Plaza,Garden,French Restaurant,Steakhouse,Art Gallery
526,LONDON,1,Coffee Shop,Café,Grocery Store,Pub,Italian Restaurant,Turkish Restaurant,Bus Stop,Supermarket,Pharmacy,Sushi Restaurant
527,LONDON,1,Pub,Grocery Store,Bus Stop,Coffee Shop,Indian Restaurant,Turkish Restaurant,Café,Fish & Chips Shop,Clothing Store,Construction & Landscaping


Cluuster 2

In [48]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,2,Grocery Store,Indian Restaurant,Train Station,Breakfast Spot,Park,Zoo Exhibit,Fish & Chips Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant


Cluster 3

In [49]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
378,"HARROW, STANMOREEDGWARE, LONDON",3,Gym,Bakery,Metro Station,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market


Cluster 4

In [50]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,4,Supermarket,Pub,Park,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Falafel Restaurant,Farmers Market
45,"BEXLEYHEATH, LONDON",4,Supermarket,Park,Historic Site,Pub,Train Station,Coffee Shop,Convenience Store,Bus Stop,Construction & Landscaping,Golf Course
124,LONDON,4,Supermarket,Park,Historic Site,Pub,Train Station,Coffee Shop,Convenience Store,Bus Stop,Construction & Landscaping,Golf Course
291,"LONDON, SIDCUP",4,Supermarket,Park,Historic Site,Pub,Train Station,Coffee Shop,Convenience Store,Bus Stop,Construction & Landscaping,Golf Course
506,LONDON,4,Supermarket,Park,Historic Site,Pub,Train Station,Coffee Shop,Convenience Store,Bus Stop,Construction & Landscaping,Golf Course


Cluster 5

In [51]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]


,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,LONDON,5,Music Venue,Bus Station,Clothing Store,Gym / Fitness Center,Supermarket,Historic Site,Fishing Store,Fast Food Restaurant,Filipino Restaurant,Film Studio


## Exploring Paris

### Neighborhoods of Paris

### Data Collection

In [52]:
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print("Data Downloaded!")

Data Downloaded!


In [53]:
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


### Data Preprocessing

In [54]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


### Feature Selection

In [55]:
df_2 = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df_2

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"
...,...,...,...,...
1295,77520,CESSOY-EN-MONTOIS,SEINE-ET-MARNE,"[48.50730730461658, 3.138844194183689]"
1296,93420,VILLEPINTE,SEINE-SAINT-DENIS,"[48.95902025378707, 2.536306342059409]"
1297,77130,CANNES-ECLUSE,SEINE-ET-MARNE,"[48.36403767307805, 2.990786679832767]"
1298,78930,VILLETTE,YVELINES,"[48.92627887061508, 1.6937417245662671]"


### Feature Engineering

In [56]:
df_paris = df_2[df_2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"
5,75019,PARIS-19E-ARRONDISSEMENT,PARIS,"[48.88686862295828, 2.384694327870042]"
6,75020,PARIS-20E-ARRONDISSEMENT,PARIS,"[48.86318677744551, 2.400819826729021]"
7,75015,PARIS-15E-ARRONDISSEMENT,PARIS,"[48.84015541860987, 2.293559372435076]"
8,75005,PARIS-5E-ARRONDISSEMENT,PARIS,"[48.844508659617546, 2.349859385560182]"
9,75007,PARIS-7E-ARRONDISSEMENT,PARIS,"[48.85608259819694, 2.312438687733857]"


In [57]:
df_paris.shape


(20, 4)

We broke down Paris data to 20 recors, much smaller than London

In [58]:
df_paris.dtypes


postal_code     object
nom_comm        object
nom_dept        object
geo_point_2d    object
dtype: object

### Geolocations of the Neighborhoods of Paris

In [59]:
df_paris['geo_point_2d'][0]


[48.87689616237872, 2.337460241388529]

In [60]:

temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.86305413181178, 2.359361058970589]
4      [48.84896809191946, 2.332670898588416]
5      [48.88686862295828, 2.384694327870042]
6      [48.86318677744551, 2.400819826729021]
7      [48.84015541860987, 2.293559372435076]
8     [48.844508659617546, 2.349859385560182]
9      [48.85608259819694, 2.312438687733857]
10     [48.87252726662346, 2.312582560420059]
11     [48.82871768452136, 2.362468228516128]
12     [48.83515623066034, 2.419807034965275]
13    [48.892735074561706, 2.348711933867703]
14      [48.8626304851685, 2.336293446550539]
15     [48.87602855694339, 2.361112904561707]
16     [48.86039876035177, 2.262099559395783]
17    [48.854228281954754, 2.357361938142205]
18     [48.88733716648682, 2.307485559493426]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [61]:
paris_latlng = df_paris['geo_point_2d'].astype('str')


### Latitude

In [62]:
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5      48.88686862295828
6      48.86318677744551
7      48.84015541860987
8     48.844508659617546
9      48.85608259819694
10     48.87252726662346
11     48.82871768452136
12     48.83515623066034
13    48.892735074561706
14      48.8626304851685
15     48.87602855694339
16     48.86039876035177
17    48.854228281954754
18     48.88733716648682
19     48.82899321160942
Name: geo_point_2d, dtype: object

### Longitude

In [63]:
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.384694327870042
6      2.400819826729021
7      2.293559372435076
8      2.349859385560182
9      2.312438687733857
10     2.312582560420059
11     2.362468228516128
12     2.419807034965275
13     2.348711933867703
14     2.336293446550539
15     2.361112904561707
16     2.262099559395783
17     2.357361938142205
18     2.307485559493426
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [64]:

paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

Preparing our combined data by dropping the geo_point_2d column from our previously stored df_paris and concatenating with the latitude and longitude extracted from it



In [65]:
paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
5,75019,PARIS-19E-ARRONDISSEMENT,PARIS,48.886869,2.384694
6,75020,PARIS-20E-ARRONDISSEMENT,PARIS,48.863187,2.400820
7,75015,PARIS-15E-ARRONDISSEMENT,PARIS,48.840155,2.293559
8,75005,PARIS-5E-ARRONDISSEMENT,PARIS,48.844509,2.349859
9,75007,PARIS-7E-ARRONDISSEMENT,PARIS,48.856083,2.312439


### Coordinates for Paris

In [66]:
paris = geocode(address='Paris, France, FR')[0]
paris_lng_coords = paris['location']['x']
paris_lat_coords = paris['location']['y']
print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geolocation of Paris:  48.85341000000005 2.3488000000000397


### Visualize the Map of Paris

In [67]:
# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

### Venues in Paris

Get nearby venues present in each neighborhood

In [69]:
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], paris_combined_data['Latitude'], paris_combined_data['Longitude'])


PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


Sample our data

In [70]:
venues_in_Paris.head()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,Plaza
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Qee,Gym / Fitness Center
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,Wine Bar
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,Vegetarian / Vegan Restaurant
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Aji Dulce,Venezuelan Restaurant


In [71]:
venues_in_Paris.shape


(1329, 5)

1329 records of venues were collected for Paris

### Grouping by Venue Categories

Check how many Venue Catergories there are 

In [72]:
venues_in_Paris.groupby('Venue Category').max()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Afghan Restaurant,PARIS-11E-ARRONDISSEMENT,48.859415,2.378741,Afghanistan
African Restaurant,PARIS-9E-ARRONDISSEMENT,48.886869,2.400820,Wally Le Saharien
American Restaurant,PARIS-19E-ARRONDISSEMENT,48.892735,2.384694,Harper's
Antique Shop,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Hôtel des Ventes Drouot
Argentinian Restaurant,PARIS-3E-ARRONDISSEMENT,48.863054,2.359361,Anahi
...,...,...,...,...
Wine Bar,PARIS-9E-ARRONDISSEMENT,48.892735,2.400820,Vingt Vins d'Art
Wine Shop,PARIS-3E-ARRONDISSEMENT,48.876029,2.361113,Trois Fois Vin
Women's Store,PARIS-6E-ARRONDISSEMENT,48.867903,2.344107,L'Appartement Sézane


With 206 records, Paris is very diverse

### One Hot Encoding

In [73]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Add Neighborhoods

In [74]:
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Venue categories mean value

In [75]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-10E-ARRONDISSEMENT,0.00000,0.02,0.0,0.0,0.0,0.0,0.00000,0.0,0.030000,...,0.0,0.0,0.010000,0.0,0.00000,0.02000,0.02,0.0,0.000000,0.000000
1,PARIS-11E-ARRONDISSEMENT,0.02381,0.00,0.0,0.0,0.0,0.0,0.02381,0.0,0.023810,...,0.0,0.0,0.047619,0.0,0.02381,0.02381,0.00,0.0,0.000000,0.000000
2,PARIS-12E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.00000,0.00000,0.00,0.0,0.142857,0.142857
3,PARIS-13E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.183333,...,0.0,0.0,0.000000,0.0,0.20000,0.00000,0.00,0.0,0.000000,0.000000
4,PARIS-14E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.00000,0.00000,0.00,0.0,0.000000,0.000000


### Top Venue Categories

In [76]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Hotel,Coffee Shop,Café,Bistro,Italian Restaurant,Indian Restaurant,Pizza Place,Asian Restaurant,Breakfast Spot
1,PARIS-11E-ARRONDISSEMENT,Restaurant,Café,French Restaurant,Italian Restaurant,Bakery,Bistro,Vegetarian / Vegan Restaurant,Cocktail Bar,Pastry Shop,Mediterranean Restaurant
2,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Bistro,Park,Monument / Landmark,Zoo,Pool,Deli / Bodega,Fountain,Food Court
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Hotel,Creperie,Dessert Shop
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Hotel,Japanese Restaurant,Food & Drink Shop,Bistro,Café,Italian Restaurant,Bakery,Brasserie,Fast Food Restaurant


### Model Building

Use K Means Clustering to cluster the city of Paris to 5 

In [77]:
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(n_clusters=5, random_state=0)

### Labeling Clustered Data

In [78]:
kmeans_Paris.labels_


array([2, 2, 3, 4, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2],
      dtype=int32)

In [79]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [80]:
paris_data = paris_combined_data

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,3,French Restaurant,Hotel,Bakery,Bistro,Japanese Restaurant,Lounge,Wine Bar,Bar,Cocktail Bar,Pizza Place
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,3,French Restaurant,Cocktail Bar,Wine Bar,Italian Restaurant,Salad Place,Thai Restaurant,Bakery,Bar,Hotel,Mexican Restaurant
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,3,Restaurant,Café,French Restaurant,Italian Restaurant,Bakery,Bistro,Vegetarian / Vegan Restaurant,Cocktail Bar,Pastry Shop,Mediterranean Restaurant
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,3,Coffee Shop,French Restaurant,Bakery,Japanese Restaurant,Burger Joint,Bistro,Art Gallery,Gourmet Shop,Cocktail Bar,Sandwich Place
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,3,French Restaurant,Chocolate Shop,Bistro,Bakery,Italian Restaurant,Athletics & Sports,Hotel,Fountain,Pastry Shop,Pub


Drop all NaN values to prevent skew

In [81]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])


### Visualize the clustered neighborhood

In [82]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

### Examining our Cluster

Cluster 1

In [83]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]


,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,PARIS-16E-ARRONDISSEMENT,1,Lake,Plaza,Pool,Boat or Ferry,Bus Station,Bus Stop,French Restaurant,Park,Art Museum,Diner


Cluster 2

In [84]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]


,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,PARIS-7E-ARRONDISSEMENT,2,French Restaurant,Hotel,Café,Italian Restaurant,Plaza,History Museum,Coffee Shop,Cocktail Bar,Art Museum,Gourmet Shop
10,PARIS-8E-ARRONDISSEMENT,2,French Restaurant,Hotel,Art Gallery,Spa,Cocktail Bar,Theater,Coffee Shop,Grocery Store,Modern European Restaurant,Furniture / Home Store
18,PARIS-17E-ARRONDISSEMENT,2,French Restaurant,Hotel,Bakery,Italian Restaurant,Café,Japanese Restaurant,Bistro,Plaza,Restaurant,Diner
19,PARIS-14E-ARRONDISSEMENT,2,French Restaurant,Hotel,Japanese Restaurant,Food & Drink Shop,Bistro,Café,Italian Restaurant,Bakery,Brasserie,Fast Food Restaurant


Cluster 3

In [85]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]


,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,3,French Restaurant,Hotel,Bakery,Bistro,Japanese Restaurant,Lounge,Wine Bar,Bar,Cocktail Bar,Pizza Place
1,PARIS-2E-ARRONDISSEMENT,3,French Restaurant,Cocktail Bar,Wine Bar,Italian Restaurant,Salad Place,Thai Restaurant,Bakery,Bar,Hotel,Mexican Restaurant
2,PARIS-11E-ARRONDISSEMENT,3,Restaurant,Café,French Restaurant,Italian Restaurant,Bakery,Bistro,Vegetarian / Vegan Restaurant,Cocktail Bar,Pastry Shop,Mediterranean Restaurant
3,PARIS-3E-ARRONDISSEMENT,3,Coffee Shop,French Restaurant,Bakery,Japanese Restaurant,Burger Joint,Bistro,Art Gallery,Gourmet Shop,Cocktail Bar,Sandwich Place
4,PARIS-6E-ARRONDISSEMENT,3,French Restaurant,Chocolate Shop,Bistro,Bakery,Italian Restaurant,Athletics & Sports,Hotel,Fountain,Pastry Shop,Pub
5,PARIS-19E-ARRONDISSEMENT,3,French Restaurant,Bar,Hotel,Bistro,Supermarket,Beer Bar,Brewery,Seafood Restaurant,Sandwich Place,Spa
6,PARIS-20E-ARRONDISSEMENT,3,Bakery,Bistro,Plaza,Japanese Restaurant,Bar,French Restaurant,Food & Drink Shop,Park,Café,Italian Restaurant
7,PARIS-15E-ARRONDISSEMENT,3,Italian Restaurant,French Restaurant,Hotel,Coffee Shop,Park,Bistro,Restaurant,Indian Restaurant,Japanese Restaurant,Lebanese Restaurant
8,PARIS-5E-ARRONDISSEMENT,3,French Restaurant,Hotel,Italian Restaurant,Bakery,Plaza,Café,Bar,Greek Restaurant,Pub,Science Museum
13,PARIS-18E-ARRONDISSEMENT,3,Bar,French Restaurant,Italian Restaurant,Pizza Place,Restaurant,Café,Bistro,Plaza,Hotel,Convenience Store


Cluster 4

In [86]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]


,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,PARIS-12E-ARRONDISSEMENT,4,Zoo Exhibit,Supermarket,Bistro,Park,Monument / Landmark,Zoo,Pool,Deli / Bodega,Fountain,Food Court


Cluster 5

In [87]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]


,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,PARIS-13E-ARRONDISSEMENT,5,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Hotel,Creperie,Dessert Shop


### Results and Discussion

London has very multicultural neighborhoods.  The cuusines range from Italian, Turkish to Chinese.  There is also a variety of different types of eating places such as restaurants, bars, coffee shops and juice bars. Further, there are numerous shopping options including fish markets, clothing stores, flea markets, glower shops etc.  for leisure, neighborhoods have parks, gold courses, gyms, zoos, etc.  Main methods of transportation are buses and trains. London would provide an entertainig experience.

Paris is much smaller geographically, but has a wide variety of activities and eateries.  The cusines offered are French, Thai, Cambodian, Asian, etc. There are many restaurants and bars, and a numerous amount of bistros.  There are more modes of public transport in paris, namely buses, bikes boats and ferries  In regards to leisure and sight seeing, there are plazas, parks, historic sites, clothing shops, art galleries and museums.  Paris seems like a very diverse interesting vacation spot. 

### Conclusion

This project was meant to showcase what the cities of Paris and London had to offer to potential migrants and tourists. We explored venues present in each location.  Each of the cities neighborhoods have a wide variety of expereinces.  Yet, they offer different experiences.  Both offer a great getaway with many places to explore and numerous cusines to try.  Culture is evident in both cities.  Our findings will help people narrow down what specific experiences they would like and which city matcher their preferences more. 